__Задача 1__

Даны значения величины заработной платы заемщиков банка (`salary`) и значения их поведенческого кредитного скоринга (`scoring`):
```
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]
```
Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек `numpy`, `pandas` и др.) найдите:
1. ковариацию (смещённую и несмещённую) этих двух величин,
2. коэффициент корреляции Пирсона.

Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.


In [1]:
import numpy as np
import pandas as pd

In [2]:
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

In [4]:
len(salary), len(scoring)

(10, 10)

In [13]:
sum_salary = 0 
sum_scoring = 0 
for i in range(len(salary)): 
    sum_salary+=salary[i]
for j in range(len(scoring)):
    sum_scoring+=scoring[j]
    
sum_salary, sum_scoring

(1014, 7099)

In [14]:
average_salary = sum_salary/len(salary)
average_scoring = sum_scoring/len(scoring)

average_salary, average_scoring

(101.4, 709.9)

In [15]:
np.mean(salary)

101.4

In [16]:
np.mean(scoring)

709.9

Несмещённую оценку можно посчитать следующим образом:
$$\sigma_{XY} = \dfrac{1}{n - 1} \displaystyle\sum_{i = 1}^n (x_i - \overline{X}) \cdot (y_i - \overline{Y})$$

Здесь $X$, $Y$ — выборки размера $n$.

In [36]:
cov = [(salary[i]-average_salary)*(scoring[i]-average_scoring)/(len(salary)-1) for i in range(len(salary))]
cov

[2278.9955555555553,
 851.6399999999999,
 1615.4733333333334,
 2290.806666666667,
 1711.6955555555555,
 -101.52666666666677,
 299.67333333333323,
 1037.34,
 74.6066666666667,
 116.67333333333328]

По аналогии с смещенной дисперсией, в делителе будет просто "n"

In [37]:
cov_sm = [(salary[i]-average_salary)*(scoring[i]-average_scoring)/len(salary) for i in range(len(salary))]
cov_sm

[2051.096,
 766.4759999999999,
 1453.926,
 2061.726,
 1540.526,
 -91.3740000000001,
 269.7059999999999,
 933.606,
 67.14600000000003,
 105.00599999999994]

In [38]:
fin_cov=0
fin_cov_sm=0
for i in range(len(cov)):
    fin_cov+=cov[i]
    fin_cov_sm+=cov_sm[i]

fin_cov, fin_cov_sm

(10175.37777777778, 9157.84)

In [33]:
np.cov(salary, scoring, ddof=1)

array([[ 3882.93333333, 10175.37777778],
       [10175.37777778, 33854.32222222]])

In [34]:
np.cov(salary, scoring, ddof=0)

array([[ 3494.64,  9157.84],
       [ 9157.84, 30468.89]])

__коэффициент корреляции Пирсона__:
$$r_{XY} = \dfrac{\sigma_{XY}}{\sigma_X \cdot \sigma_Y}$$

Здесь $\sigma_X$, $\sigma_Y$ — среднее квадратическое отклонение.

In [71]:
np.std(salary, ddof=1), np.std(scoring, ddof=1)

(62.31318747531162, 183.99544076477068)

In [72]:
np.corrcoef(salary, scoring)

array([[1.        , 0.88749009],
       [0.88749009, 1.        ]])

In [74]:
rxy=fin_cov/(np.std(salary, ddof=1)*np.std(scoring, ddof=1))
rxy

0.8874900920739164

In [75]:
sigma_scoring_array = [((scoring[i]-average_scoring)**2) for i in range(len(scoring))]
sigma_salary_array  = [((salary[i]-average_salary)**2) for i in range(len(salary))]

sigma_sum_scoring = 0 
sigma_sum_salary = 0

for i in range(len(scoring)):
    sigma_sum_scoring+=sigma_scoring_array[i]

for j in range(len(salary)):
    sigma_sum_salary+=sigma_salary_array[j]
    
sigma_sum_salary, sigma_sum_scoring

(34946.4, 304688.9000000001)

In [79]:
sigma_salary=((sigma_sum_salary)/(len(salary)-1))**0.5
sigma_scoring=((sigma_sum_scoring)/(len(scoring)-1))**0.5
sigma_salary, sigma_scoring

(62.31318747531162, 183.99544076477068)

In [80]:
rxy_initial=fin_cov/(sigma_salary*sigma_scoring)
rxy_initial

0.8874900920739164

__Задача 2__

Проведите тест на значимость коэффициента корреляции Пирсона, найденного в предыдущей задаче. Что для этого нужно знать:
* Нулевая гипотеза: реальный коэффициент корреляции равен 0. Альтернативная гипотеза двухсторонняя.
* Статистика: `t = r * sqrt(n - 2) / sqrt(1 - r ** 2)`, где `r` - коэффициент корреляции Пирсона, посчитанный по выборке.
* В предположении верности нулевой гипотезы эта статистика имеет распределение Стьюдента с параметром `df = n - 2`.


In [81]:
from scipy import stats
import math
n=10
df=n-2
alpha=0.05


In [82]:
t1 = stats.t.ppf(alpha / 2, df=n - 2)
t2 = stats.t.ppf(1 - alpha / 2, df=n - 2)
t1, t2

(-2.306004135033371, 2.3060041350333704)

Критическая область: 
$$\Omega_\alpha = (-\infty, -2.306) \cup (2.306, \infty)$$

In [83]:
stat = rxy_initial*((n - 2)**0.5) / ((1 - rxy_initial**2)**0.5)
stat

5.447168150485581

Статистический тест попал в критическую область, следовательно, гипотеза неверна. 

__Задача 3__

Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:
```
131, 125, 115, 122, 131, 115, 107, 99, 125, 111
```
Известно, что в генеральной совокупности IQ распределен нормально. Найдите доверительный интервал для математического ожидания с надежностью 0.95.

In [84]:
iq=np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111])

In [85]:
alpha=0.05

In [86]:
iq_mean = iq.mean()

In [87]:
iq_std = iq.std(ddof=1)

In [88]:
t1 = stats.t.ppf(alpha / 2, iq.shape[0] - 1)
t2 = stats.t.ppf(1 - alpha / 2, iq.shape[0] - 1)

t1, t2

(-2.262157162740992, 2.2621571627409915)

доверительный интервал: 

In [89]:
(iq_mean + t1 * iq_std / np.sqrt(n), iq_mean + t2 * iq_std / np.sqrt(n))

(110.55608365158724, 125.64391634841274)